### Student Information
Name:

Student ID:

GitHub ID:

Kaggle name:

Kaggle private scoreboard snapshot:

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home exercises** in the [DM2024-Lab2-master Repo](https://github.com/didiersalazar/DM2024-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook.


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework) regarding Emotion Recognition on Twitter by this link: https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework. The scoring will be given according to your place in the Private Leaderboard ranking:
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (0.6N + 1 - x) / (0.6N) * 10 + 20 points, where N is the total number of participants, and x is your rank. (ie. If there are 100 participants and you rank 3rd your score will be (0.6 * 100 + 1 - 3) / (0.6 * 100) * 10 + 20 = 29.67% out of 30%.)   
    Submit your last submission **BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)**. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developing the model for the competition (You can use code and comment on it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained.


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)__.

# Data

In [1]:
!mkdir -p ~/.kaggle
!mv /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c dm-2024-isa-5810-lab-2-homework
!unzip -q dm-2024-isa-5810-lab-2-homework.zip

 97% 145M/149M [00:02<00:00, 81.7MB/s]
100% 149M/149M [00:02<00:00, 75.3MB/s]


In [2]:
import pandas as pd

data_ids = pd.read_csv('data_identification.csv')
data_ids.head()

,tweet_id,identification
0,0x28cc61,test
1,0x29e452,train
2,0x2b3819,train
3,0x2db41f,test
4,0x2a2acc,train


In [3]:
train_data_ids = data_ids[data_ids['identification'] == 'train'].drop(['identification'], axis=1)
test_data_ids = data_ids[data_ids['identification'] == 'test'].drop(['identification'], axis=1)

train_data_ids.head()

,tweet_id
1,0x29e452
2,0x2b3819
4,0x2a2acc
5,0x2a8830
6,0x20b21d


In [4]:
emotion_labels = pd.read_csv('emotion.csv')
emotion_labels.head()

,tweet_id,emotion
0,0x3140b1,sadness
1,0x368b73,disgust
2,0x296183,anticipation
3,0x2bd6e1,joy
4,0x2ee1dd,anticipation


In [5]:
tweets_df = pd.read_json('tweets_DM.json', lines=True)

# Use json_normalize to flatten only the `_source` column and keep other columns intact
source_df = pd.json_normalize(tweets_df['_source'])

# Concatenate the flattened `_source` columns with the original dataframe
tweets_df = pd.concat([tweets_df.drop(columns=['_source']), source_df], axis=1)

tweets_df.head()

,_score,_index,_crawldate,_type,tweet.hashtags,tweet.tweet_id,tweet.text
0,391,hashtag_tweets,2015-05-23 11:42:47,tweets,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,433,hashtag_tweets,2016-01-28 04:52:09,tweets,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,232,hashtag_tweets,2017-12-25 04:39:20,tweets,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k..."
3,376,hashtag_tweets,2016-01-24 23:53:05,tweets,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,989,hashtag_tweets,2016-01-08 17:18:59,tweets,[],0x2de201,"""Trust is not the same as faith. A friend is s..."


In [6]:
train_data = pd.merge(train_data_ids, emotion_labels, on='tweet_id', how='inner')
train_data = pd.merge(train_data, tweets_df, left_on='tweet_id', right_on='tweet.tweet_id', how='inner')
train_data.head()

,tweet_id,emotion,_score,_index,_crawldate,_type,tweet.hashtags,tweet.tweet_id,tweet.text
0,0x29e452,joy,809,hashtag_tweets,2015-01-17 03:07:03,tweets,[],0x29e452,Huge Respect🖒 @JohnnyVegasReal talking about l...
1,0x2b3819,joy,808,hashtag_tweets,2016-07-02 09:34:06,tweets,"[spateradio, app]",0x2b3819,Yoooo we hit all our monthly goals with the ne...
2,0x2a2acc,trust,16,hashtag_tweets,2016-08-15 18:18:39,tweets,[],0x2a2acc,@KIDSNTS @PICU_BCH @uhbcomms @BWCHBoss Well do...
3,0x2a8830,joy,768,hashtag_tweets,2017-02-11 08:49:46,tweets,"[PUBG, GamersUnite, twitch, BeHealthy, StayPos...",0x2a8830,Come join @ambushman27 on #PUBG while he striv...
4,0x20b21d,anticipation,70,hashtag_tweets,2016-11-23 05:37:10,tweets,"[strength, bones, God]",0x20b21d,@fanshixieen2014 Blessings!My #strength little...


In [7]:
test_data = pd.merge(test_data_ids, tweets_df, left_on='tweet_id', right_on='tweet.tweet_id', how='inner')
test_data.head()

,tweet_id,_score,_index,_crawldate,_type,tweet.hashtags,tweet.tweet_id,tweet.text
0,0x28cc61,107,hashtag_tweets,2017-01-17 14:13:32,tweets,[],0x28cc61,@Habbo I've seen two separate colours of the e...
1,0x2db41f,728,hashtag_tweets,2015-10-17 06:46:20,tweets,[],0x2db41f,@FoxNews @KellyannePolls No serious self respe...
2,0x2466f6,491,hashtag_tweets,2016-12-19 03:50:27,tweets,[womendrivers],0x2466f6,"Looking for a new car, and it says 1 lady owne..."
3,0x23f9e9,28,hashtag_tweets,2017-04-09 19:32:19,tweets,[robbingmembers],0x23f9e9,@cineworld “only the brave” just out and fount...
4,0x1fb4e1,925,hashtag_tweets,2016-01-15 11:59:31,tweets,[],0x1fb4e1,Felt like total dog 💩 going into open gym and ...


# Data preprocessing

In [13]:
from sklearn.model_selection import train_test_split

X = train_data['tweet.text']
y = train_data['emotion']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Model training

In [8]:
!pip install torch transformers

In [14]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import TextClassificationPipeline
from sklearn.preprocessing import LabelEncoder


# Load the pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"  # You can also try "bertweet-base" for Twitter-specific tasks

# Define the BERT model for sequence classification
class EmotionClassifier(nn.Module):
    def __init__(self, num_labels):
        super(EmotionClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        # Get hidden states from BERT
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # Apply dropout and classification layer on the pooled output
        pooled_output = self.dropout(outputs.pooler_output)
        return self.classifier(pooled_output)

In [15]:
num_labels = 8
model = EmotionClassifier(num_labels=num_labels)
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
# Encode labels (fit the label encoder only on training data to simulate real-world training)
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

# Encode the text data
X_train_encoded = tokenizer(list(X_train), truncation=True, padding=True, max_length=128)
X_val_encoded = tokenizer(list(X_val), truncation=True, padding=True, max_length=128)

In [ ]:
# Dataset class for loading text data from DataFrame
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# Initialize datasets
train_dataset = TweetDataset(X_train_encoded, y_train_encoded)
val_dataset = TweetDataset(X_val_encoded, y_val_encoded)

In [ ]:
# Sample training configuration
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Training with Hugging Face Trainer API
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Train the model
trainer.train()

# Use the model for predictions
classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, framework="pt")
sample_tweet = "I'm so excited about this new feature!"
pred = classifier(sample_tweet)
print(pred)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluates at the end of each epoch
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize the Trainer with the training and validation datasets
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Start training
trainer.train()